In [9]:
import torch

## Named Tensors

In [10]:
img_t = torch.randn(3, 5, 5) # shape [channels, rows, columns]
batch_t = torch.randn(2, 3, 5, 5) # shape [batch, channels, rows, columns]

In [11]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])
weights_named

tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [12]:
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print("img named:", img_named.shape, img_named.names)
print("batch named:", batch_named.shape, batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [13]:
weights_aligned = weights_named.align_as(img_named)
print(weights_aligned.shape, weights_aligned.names)
weights_aligned

torch.Size([3, 1, 1]) ('channels', 'rows', 'columns')


tensor([[[0.2126]],

        [[0.7152]],

        [[0.0722]]], names=('channels', 'rows', 'columns'))

In [14]:
gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

In [15]:
# cant combine dimensions with different names
# (img_named[..., :3] * weights_named).sum('channels')
# RuntimeError: Error when attempting to broadcast dims ['channels', 'rows', 'columns'] and dims ['channels']:
# dim 'columns' and dim 'channels' are at the same position from the right but do not match.


In [16]:
gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))

## Tensor API

In [29]:
some_t = torch.ones(3, 4, 5)
transpose_t = some_t.transpose(0, 2)
some_t.shape, transpose_t.shape

(torch.Size([3, 4, 5]), torch.Size([5, 4, 3]))

In [30]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_storage = points.storage()
points_storage

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

In [31]:
points.zero_()
points

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [32]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]

In [33]:
points.storage(), points.storage_offset(), points.stride(), points.shape

( 4.0
  1.0
  5.0
  3.0
  2.0
  1.0
 [torch.FloatStorage of size 6],
 0,
 (2, 1),
 torch.Size([3, 2]))

In [34]:
second_point.storage(), second_point.storage_offset(), second_point.stride(), second_point.shape

( 4.0
  1.0
  5.0
  3.0
  2.0
  1.0
 [torch.FloatStorage of size 6],
 2,
 (1,),
 torch.Size([2]))

In [35]:
second_point[0] = 10
points, second_point

(tensor([[ 4.,  1.],
         [10.,  3.],
         [ 2.,  1.]]),
 tensor([10.,  3.]))

In [36]:
third_point = points[2].clone()
third_point.storage(), third_point.storage_offset(), third_point.stride()

( 2.0
  1.0
 [torch.FloatStorage of size 2],
 0,
 (1,))

In [37]:
third_point[1] = 10
points, third_point

(tensor([[ 4.,  1.],
         [10.,  3.],
         [ 2.,  1.]]),
 tensor([ 2., 10.]))

In [51]:
points_t = points.T

In [52]:
points.is_contiguous(), points_t.is_contiguous()

(True, False)

In [53]:
points_t_cont = points_t.contiguous()
print(points_t.shape, points_t_cont.shape)
points_t.stride(), points_t_cont.stride()

torch.Size([2, 3]) torch.Size([2, 3])


((1, 2), (3, 1))

## Moving Tensors

In [61]:
points_gpu = torch.tensor([[4.0, 1.0], [10.0, 3.0], [2.0, 1.0]], device='cuda')
points_t_gpu = points_t.to('cuda')

In [56]:
points * 2

tensor([[ 8.,  2.],
        [20.,  6.],
        [ 4.,  2.]])

In [62]:
points_gpu * 2

tensor([[ 8.,  2.],
        [20.,  6.],
        [ 4.,  2.]], device='cuda:0')

In [64]:
# cant do operations with tensors on different devices
# points + points_gpu
# RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [70]:
points_np = points.numpy()
points_np, type(points_np), type(points)

(array([[ 4.,  1.],
        [10.,  3.],
        [ 2.,  1.]], dtype=float32),
 numpy.ndarray,
 torch.Tensor)

## Serializing Tensors

In [73]:
torch.save(points, '../data/02/points.tensor')

In [74]:
saved_points = torch.load('../data/02/points.tensor')
saved_points

tensor([[ 4.,  1.],
        [10.,  3.],
        [ 2.,  1.]])

In [78]:
import h5py

In [81]:
with h5py.File('../data/02/points.hdf5', 'w') as f:
    f.create_dataset('coords', data=points_gpu.to('cpu').numpy())

In [93]:
with h5py.File('../data/02/points.hdf5', 'r') as f:
    last_points = f['coords'][:]
    last_points = torch.from_numpy(last_points)
    last_points = last_points.to('cuda')

In [94]:
last_points

tensor([[ 4.,  1.],
        [10.,  3.],
        [ 2.,  1.]], device='cuda:0')